In [1]:
!pip install -q -U sentence-transformers faiss-cpu peft huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json, faiss, torch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from peft import PeftModel
from huggingface_hub import hf_hub_download

In [3]:
BASE_MODEL = "Qwen/Qwen3-Embedding-0.6B"
ADAPTER_REPO = "Isaac74/qwen3-0.6b-lightweight-semantic-mathlib-search-adapter"
HNSW_IDX_FILENAME = "mathlib420_final_hnsw.idx"
PUBMAP_FILENAME   = "mathlib420_final_public_index.jsonl"

INDEX_PATH = Path(hf_hub_download(
    repo_id=ADAPTER_REPO,
    filename=f"faiss/{HNSW_IDX_FILENAME}",
    repo_type="model"
))
PUBMAP_PATH_JSONL = Path(hf_hub_download(
    repo_id=ADAPTER_REPO,
    filename=f"faiss/{PUBMAP_FILENAME}",
    repo_type="model"
))

# --- load FAISS index (kept) ---
index = faiss.read_index(str(INDEX_PATH))
index.hnsw.efSearch = 128

# --- load base encoder + LoRA adapter ---
model = SentenceTransformer(BASE_MODEL, trust_remote_code=True)
model.max_seq_length = 256
hf_model = model._first_module().auto_model
peft_model = PeftModel.from_pretrained(
    hf_model,
    ADAPTER_REPO,
    subfolder="adapter",
    is_trainable=False
)
model._first_module().auto_model = peft_model
model.eval()

# --- load public mapping ---
idx2meta = {}
with PUBMAP_PATH_JSONL.open("r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        d = json.loads(line)
        idx2meta[int(d["idx"])] = (d["full_name"], d["statement"])

# --- warmup ---
_ = model.encode(["warmup"], convert_to_numpy=True, normalize_embeddings=True)
import numpy as np
dummy = np.zeros((1, index.d), dtype="float32")
index.search(dummy, 1)

print("Search ready")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mathlib420_final_hnsw.idx:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

mathlib420_final_public_index.jsonl:   0%|          | 0.00/121M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

Search ready


In [6]:
import re

def normalize_math_symbols(text: str) -> str:
    replacements = {
        "<=": "≤", ">=": "≥", "!=": "≠",
        "=>": "⇒", "<=>": "⇔", "->": "⇒",
        "÷": "/"
    }
    for old, new in sorted(replacements.items(), key=lambda kv: -len(kv[0])):
        text = text.replace(old, new)
    text = re.sub(r"\bpi\b", " π ", text, flags=re.IGNORECASE)

    ops = sorted([
        "≤","≥","≠","⇒","→","←","⇔",
        "+","-","*","/","=","<",">",
        "⊂","⊆","⊄","⊇","⊃","∈","∉","∪","∩","⊥","∥","∅",
        "∀","∃","¬","~","∧","∨","±","√","∑","∏","≈","≅","≃","≡"
    ], key=len, reverse=True)
    pattern = re.compile(r'\s*(' + '|'.join(map(re.escape, ops)) + r')\s*')
    norm = pattern.sub(r' \1 ', text)
    return re.sub(r'\s{2,}', ' ', norm).strip()

def unique_top_k(query: str, k: int = 10, search_k: int = 50):
    q_norm = normalize_math_symbols(query)
    q_vec = model.encode([q_norm], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    D, I = index.search(q_vec, search_k)
    seen, out = set(), []
    for score, idx in zip(D[0], I[0]):
        meta = idx2meta.get(int(idx))
        if meta is None:
            continue
        full_name, stmt = meta
        if full_name in seen:
            continue
        seen.add(full_name)
        out.append((full_name, stmt, float(score)))
        if len(out) == k:
            break
    return out

# --- interactive search loop ---
print("\nEnter a natural language/formula-based search query (type 'exit' to quit)\n" + "-"*60)
while True:
    q = input("🔎  ")
    if q.strip().lower() in {"exit", "quit"}:
        break
    results = unique_top_k(q, k=10, search_k=50)
    for full_name, stmt, score in results:
        print(f"{full_name}\n{stmt}\n")
    print("-"*60)


Enter a natural language/formula-based search query (type 'exit' to quit)
------------------------------------------------------------
🔎  w+-j=w-j for int
Int.add_neg_eq_sub
theorem add_neg_eq_sub {a b : Int} : a + -b = a - b :=

USize.add_neg_eq_sub
theorem USize.add_neg_eq_sub {a b : USize} : a + -b = a - b :=

ISize.add_neg_eq_sub
theorem ISize.add_neg_eq_sub {a b : ISize} : a + -b = a - b :=

Mathlib.Tactic.RingNF.add_neg
theorem add_neg {R} [Ring R] (a b : R) : a + -b = a - b :=

Int.sub_neg
theorem sub_neg (a b : Int) : a - -b = a + b :=

Int8.add_neg_eq_sub
theorem Int8.add_neg_eq_sub {a b : Int8} : a + -b = a - b :=

Int.neg_sub
theorem neg_sub (a b : Int) : -(a - b) = b - a :=

Int.sub_eq_add_neg
theorem sub_eq_add_neg {a b : Int} : a - b = a + -b :=

Int8.sub_neg
theorem Int8.sub_neg {a b : Int8} : a - -b = a + b :=

Int32.add_neg_eq_sub
theorem Int32.add_neg_eq_sub {a b : Int32} : a + -b = a - b :=

------------------------------------------------------------
🔎  law of cos
